IMPORTING ALL THE REQUIRED MODULES AND LIBRARIES 

In [1]:
import pandas as pd
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LinearRegression as lr
from sklearn.linear_model import ElasticNet
from lightgbm import LGBMModel
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, make_scorer

DATA PRE-PROCESSING :

READING OUR DATASET
SEPEARTING THE TARGET COLUMN IN A VARIABLE
DROPPING SOME OF THE DEPENDENT COLUMNS

In [2]:
df = pd.read_csv("recs2009_public.csv")
y = df['KWH']
X=df.drop(['DOEID','KWH','REGIONC','REPORTABLE_DOMAIN','HDD65','CDD65','YEARMADE'],axis=1)
X.shape

/home/sri/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (717,718) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(12083, 933)

SEPERATING THE FEATURES IN THREE LISTS FOR PRE-PROCESSING

In [3]:
categorical=[]
numerical = []
num_categorical=[]
for col in X.columns:
    if col == 'NWEIGHT' : continue
    if X[col].dtype=='object':
        categorical.append(col)
    elif X[col].nunique()< 10:
        num_categorical.append(col)
    else :
        numerical.append(col)

print(len(numerical))
print(len(categorical))
print(len(num_categorical))

135
5
792


CHECKING IF THERE ARE ANY MISSING VALUES.
THE SUPPORTING DOCUMENTS SAYS THE MISSING VALUES WERE IMPUTED, SO THIS WAS JUST TRUTH CHECKING!

In [4]:
X.isna().any().all()

False

APPLYING LABEL ENCODING TO "object" TYPE FEATURES (THE FEATURES IN THE LIST NAMED "categorical")

In [5]:
for i in categorical:
    le = LabelEncoder()
    X[i] = X[i].apply(str)
    print(X[i].unique())
    X[i]=le.fit_transform(X[i])
    print(X[i].unique())

['METRO' 'MICRO' 'NONE']
[0 1 2]
['U' 'R']
[1 0]
['-2' '1' '6' '15' '4' '20' '.' '12' '10' '3' '24' '55' '8' '28' '5' '16'
 '2' '40' '25']
[ 0  2 17  5 13  8  1  4  3 12  9 16 18 11 15  6  7 14 10]
['-2' '1' '5' '10' '15' '8' '.' '3' '2' '77' '30' '55' '20']
[ 0  2  9  3  4 12  1  7  5 11  8 10  6]
['4A' '3C' '5A' '6A-6B' '1A-2A' '3B-4B' '3A' '5B-5C' '7A-7B-7AK-8AK' '2B'
 '4C']
[ 5  4  7  9  0  3  2  8 10  1  6]


SPLITTING THE DATA INTO TRAIN(80%) & TEST (20%).
GETTING WEIGHTS FOR TRAIN TEST AND ENTIRE SET (WE WILL USE THESE WEIGHTS DURING TRAINING & FINDING TEST MSE).
DROPPING "NWEIGHT" FROM ALL THE INPUTS.

In [6]:
Xtr,Xte,ytr,yte = train_test_split(X,y)
train_wts = Xtr['NWEIGHT']
test_wts = Xte['NWEIGHT']
total_wts = X['NWEIGHT']
X.drop('NWEIGHT',axis=1,inplace=True)
Xtr.drop('NWEIGHT',axis=1,inplace=True)
Xte.drop('NWEIGHT',axis=1,inplace=True)

/home/sri/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3930: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


APPLYING ROBUST SCALER TO NUMERICAL FEATURES IN TRAIN SET.
USING ABOVE MEDIAN & RANGE,TRANSFORMING THE NUMERICAL FEATURES IN TEST SET.
WE CAN OBSERVE THE MEDIAN VALUES OF TRAIN SET NUMERICAL FEATURES BEFORE & AFTER SCALING.

In [7]:
rs = RobustScaler()
print("Medians before scaling:\n", Xtr[numerical].median())
Xtr[numerical]=rs.fit_transform(Xtr[numerical])
print("\n Medians after scaling:\n",Xtr[numerical].median())
Xte[numerical]=rs.transform(Xte[numerical])

/home/sri/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/sri/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Medians before scaling:
 DIVISION            5.0
HDD30YR          4473.5
CDD30YR          1190.0
NUMFLRS            -2.0
NUMAPTS            -2.0
BEDROOMS            3.0
OTHROOMS            3.0
TOTROOMS            6.0
MONRFRI2           -2.0
MONRFRI3           -2.0
TVCOLOR             2.0
NUMPC               1.0
EQUIPNOHEAT        -2.0
EQUIPM              3.0
FUELHEAT            1.0
HEATROOM            5.0
TEMPHOME           70.0
TEMPGONE           68.0
TEMPNITE           68.0
ACROOMS             4.0
TEMPHOMEAC         -2.0
TEMPGONEAC         -2.0
TEMPNITEAC         -2.0
NUMBERAC           -2.0
NUMCFAN             2.0
LGT12               0.0
LGT12EE            -2.0
LGT4                1.0
LGT4EE              0.0
LGT1                2.0
                 ...   
GALLONKERSPH        0.0
GALLONKEROTH        0.0
BTUKER              0.0
BTUKERSPH           0.0
BTUKEROTH           0.0
DOLLARKER           0.0
DOLKERSPH           0.0
DOLKEROTH           0.0
BTUWOOD             0.0
CORDSWD        

/home/sri/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/sri/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


MODEL BUILDING FOLLOWED BY RESULTS :

CREATING OBJECTS OF 3 DIFFERENT REGRESSORS.
DEFINING A SCORER FUCTION,LATER USED IN GRID SEARCH OBJECT

In [8]:
reg1= LGBMModel(objective='regression')
reg2= ElasticNet()
reg3=lr(n_jobs= -1)

def MSE(y_true,y_pred):
    mse = mean_squared_error(y_true, y_pred)
    return mse

def scorer():
    return make_scorer(MSE, greater_is_better=False)


CREATING GRID OF HYPERPARAMETERS FOR LGBM.
PERFORMING GRID SEARCH ON LGBM REGRESSOR OBJECT CREATED BEFORE

In [9]:
param_lgb={'num_leaves':[15, 31, 63, 127, 255],'bagging_fraction' :[0.6,0.7,0.8,0.9],'feature_fraction':[ 0.5,0.7,0.9],'learning_rate':[0.10, 0.03334, 0.01, 0.003334]}

gscv1 = GridSearchCV(reg1,param_lgb,scoring=scorer(),n_jobs=-1,cv=5)
gscv1.fit(Xtr,ytr,train_wts)

/home/sri/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/home/sri/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LGBMModel(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
     learning_rate=0.1, max_depth=-1, min_child_samples=20,
     min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
     n_jobs=-1, num_leaves=31, objective='regression', random_state=None,
     reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
     subsample_for_bin=200000, subsample_freq=0),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'num_leaves': [15, 31, 63, 127, 255], 'bagging_fraction': [0.6, 0.7, 0.8, 0.9], 'feature_fraction': [0.5, 0.7, 0.9], 'learning_rate': [0.1, 0.03334, 0.01, 0.003334]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(MSE, greater_is_better=False), verbose=0)

CREATING GRID OF HYPERPARAMETERS FOR ELASTIC NET.
PERFORMING GRID SEARCH ON ELASTIC NET OBJECT CREATED BEFORE

In [11]:
param_enet={'l1_ratio':[0.05,0.1,0.5,1],'alpha':[0.01,0.1,1,10],'max_iter':[1000,3000,5000]}

gscv2 = GridSearchCV(reg2,param_enet,scoring=scorer(),n_jobs=-1,cv=5)
gscv2.fit(Xtr,ytr,train_wts)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'l1_ratio': [0.05, 0.1, 0.5, 1], 'alpha': [0.01, 0.1, 1, 10], 'max_iter': [1000, 3000, 5000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(MSE, greater_is_better=False), verbose=0)

CREATING GRID OF HYPERPARAMETERS FOR LINEAR REGRESSION.
PERFORMING GRID SEARCH ON LINEAR REGRESSION OBJECT CREATED BEFORE

In [12]:
param_lr={'fit_intercept':[True,False]}

gscv3 = GridSearchCV(reg3,param_lr,scoring=scorer(),n_jobs=-1,cv=5,refit=True)
gscv3.fit(Xtr,ytr,train_wts)

/home/sri/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'fit_intercept': [True, False]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(MSE, greater_is_better=False), verbose=0)

MAKING PREDICTIONS ON TEST DATA USING THE 3 MODELS GIVEN BY GRID SEARCH
PRINTING TEST MSE ON EACH OF THESE 3 MODELS

In [13]:
y_pred_lgb=gscv1.best_estimator_.predict(Xte)
y_pred_enet=gscv2.best_estimator_.predict(Xte)
y_pred_lr=gscv3.best_estimator_.predict(Xte)

print("TEST DATA MSE FOR LGBM : ",mean_squared_error(yte.values,y_pred_lgb,test_wts))
print("TEST DATA MSE FOR ELASTIC NET : ",mean_squared_error(yte.values,y_pred_enet,test_wts))
print("TEST DATA MSE FOR LINEAR REGRESSION : ",mean_squared_error(yte.values,y_pred_lr,test_wts))

TEST DATA MSE FOR LGBM :  3605888.8798391502
TEST DATA MSE FOR ELASTIC NET :  2.6254093955372597
TEST DATA MSE FOR LINEAR REGRESSION :  0.00010478564554436469


PRINTING CV-MSE SCORE OF THE BEST ESTIMATOR OBTAINED FROM GRID SEARCH. THESE VALUES WILL BE NEGATIVE BECAUSE UNIFIED SCORING API ALWAYS MAXIMIZES THE OBJECTIVE FUNCTION.
HENCE, THESE SCORES ARE NEGATED TO GET THE ACTUAL VALUES.

In [16]:
print("BEST CV-MSE SCORE FOR LGBM : ", -gscv1.best_score_)
print("BEST CV-MSE SCORE FOR ELASTIC NET : ", -gscv2.best_score_)
print("BEST CV-MSE SCORE FOR LINEAR REGRESSION : ",-gscv3.best_score_)

BEST CV-MSE SCORE FOR LGBM :  196670.27911505874
BEST CV-MSE SCORE FOR ELASTIC NET :  2.3066801889158293
BEST CV-MSE SCORE FOR LINEAR REGRESSION :  5161.859224101664


Printing the first 15 most important features (if the selected model is Elastic Net )with their coefficient values.
The argsort() performs just ascending sort, but we need descending sort and hence the negatives of the coefficients are sorted.

In [62]:
import numpy as np
feats_coef = gscv2.best_estimator_.coef_
index=(-feats_coef).argsort()[:]
print("The 15 most discriminative features with their coefficients are: \n",)
for i in range(15):
    print(X.columns[index[i]],":",feats_coef[index[i]])

The 15 most discriminative features with their coefficients are: 

DesignDBT1 : 4442.72597075642
DesignDBT99 : 1912.999187832921
GWT : 1745.0518409096235
OA_LAT : 1268.8036239207518
WSF : 1108.054759135953
GND_HDD65 : 734.4394256472076
CDD80 : 0.2847943995461545
HDD50 : 0.03513083829426013
IECC_Climate_Pub : 0.00865240427845286
SCALEKER : 0.00752762818688332
PERIODKR : 0.004894771045537108
SCALEFO : 0.004148833226539084
PERIODFO : 0.0010417489387493854
SCALELP : 0.0009605780126782693
PERIODLP : 1.9495888489760106e-05
